In [1]:
# Prepare data

import pandas as pd
df = pd.read_csv('anomaly.csv')

print(df.groupby('Quality')['Quality'].count())



Quality
Bad      407
Good    1054
Name: Quality, dtype: int64


In [2]:
# Drop useless columns

df.drop(['Date'], axis=1, inplace=True)
print(df.head())

# If there are missing entries, drop them. (drop null data)
df.dropna(inplace=True,axis=1)


# Convert non-numeric to numeric
df.Quality[df.Quality == 'Good'] = 1
df.Quality[df.Quality == 'Bad'] = 2

print("###############")
print(df.head())

   Power  Detector Quality
0     96         8    Good
1     96        10    Good
2     91         8    Good
3     97         9    Good
4     91        11    Good
###############
   Power  Detector Quality
0     96         8       1
1     96        10       1
2     91         8       1
3     97         9       1
4     91        11       1


<ipython-input-2-4848024c1a31>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Good'] = 1
<ipython-input-2-4848024c1a31>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Bad'] = 2


In [3]:
good_mask = df['Quality']== 1 #All good to be True for good data points
bad_mask = df['Quality']== 2 #All values False for good data points
print(good_mask.head(10))

df.drop('Quality',axis=1,inplace=True)

df_good = df[good_mask]
df_bad = df[bad_mask]

print(f"Good count: {len(df_good)}")
print(f"Bad count: {len(df_bad)}")


0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: Quality, dtype: bool
Good count: 1054
Bad count: 407


In [4]:
# Take values from df

x_good = df_good.values
x_bad = df_bad.values

from sklearn.model_selection import train_test_split

x_good_train, x_good_test = train_test_split(x_good, test_size=0.25, random_state=42)

print(f"Good train count: {len(x_good_train)}")
print(f"Good test count: {len(x_good_test)}")


Good train count: 790
Good test count: 264


In [5]:
######### 
# Define the autoencoder model with dense layers
# Train~

from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(10, input_dim=x_good.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(x_good.shape[1])) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

model.fit(x_good_train,x_good_train,verbose=1,epochs=100)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
 dense_2 (Dense)             (None, 10)                40        
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25/25 [==============================] - 0s 1ms/step - loss: 2745.7446
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 1816.1367
Epoch 3/100
25/25 [=====

25/25 [==============================] - 0s 1ms/step - loss: 1.4463
Epoch 91/100
25/25 [==============================] - 0s 1ms/step - loss: 1.4505
Epoch 92/100
25/25 [==============================] - 0s 1ms/step - loss: 1.4308
Epoch 93/100
25/25 [==============================] - 0s 1ms/step - loss: 1.4274
Epoch 94/100
25/25 [==============================] - 0s 1ms/step - loss: 1.4212
Epoch 95/100
25/25 [==============================] - 0s 1ms/step - loss: 1.4198
Epoch 96/100
25/25 [==============================] - 0s 1ms/step - loss: 1.4140
Epoch 97/100
25/25 [==============================] - 0s 1ms/step - loss: 1.4086
Epoch 98/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3950
Epoch 99/100
25/25 [==============================] - 0s 2ms/step - loss: 1.3852
Epoch 100/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3789


In [6]:
model.fit(x_good_train,x_good_train,verbose=1,epochs=100)

pred = model.predict(x_good_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_good_test))

pred = model.predict(x_good)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_good))

pred = model.predict(x_bad)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_bad))

print(f"Insample Good Score (RMSE): {score1}".format(score1))
print(f"Out of Sample Good Score (RMSE): {score2}")
print(f"Bad sample Score (RMSE): {score3}")


Epoch 1/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3820
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3744
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3634
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 1.3594
Epoch 5/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3414
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 1.3362
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 1.3320
Epoch 8/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3168
Epoch 9/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3114
Epoch 10/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3259
Epoch 11/100
25/25 [==============================] - 0s 1ms/step - loss: 1.3030
Epoch 12/100
25/25 [==============================] - 0s 1ms/step - loss: 1.2839
Epoch 13/100
25/25 [=================

13/13 [==============================] - 0s 1ms/step
Insample Good Score (RMSE): 0.6300432010552756
Out of Sample Good Score (RMSE): 0.6298362836534576
Bad sample Score (RMSE): 1.5453658114694622


## Use the good data to train the model. When we feed bad data to the trained model, the higher score (Large reconstruction error) can be gotten from the model.